# Using the Panoptes Aggregation Tool from Zooniverse
https://aggregation-caesar.zooniverse.org/Scripts.html#scripts


In [2]:
from zooniverse.config import get_config
import pandas as pd
from pathlib import Path

### use either the subset of the subset
#phase_tag = "Iguanas 1st launch"
#data_folder = "./data/phase_1"

phase_tag = "Iguanas 2nd launch"
data_folder = "./data/phase_2"

#phase_tag = "Iguanas 3rd launch"
#data_folder = "./data/phase_3"

workflow_id_p1 = 14370.0
workflow_id_p2 = 20600.0
workflow_id_p3 = 22040.0

input_path = Path("/Users/christian/data/zooniverse")

# use_gold_standard_subset = "expert" # Use the expert-GS-Xphase as the basis
output_path = Path("/Users/christian/data/zooniverse/2024_04_16_analysis").joinpath(phase_tag).resolve()

output_plot_path = output_path.joinpath("plots")
output_plot_path.mkdir(parents=True, exist_ok=True)

reprocess = False

config = get_config(phase_tag=phase_tag, input_path=input_path, output_path=output_path)
config


{'annotations_source': PosixPath('/Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv'),
 'goldstandard_data': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/expert-GS-2ndphase.csv'),
 'gold_standard_image_subset': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/2-T2-GS-results-5th-0s.csv'),
 'image_source': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/2nd launch_without_prefix'),
 'yes_no_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_16_analysis/Iguanas 2nd launch/yes_no_dataset_Iguanas 2nd launch.csv'),
 'flat_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_16_analysis/Iguanas 2nd launch/flat_dataset_Iguanas 2nd launch.csv'),
 'flat_panoptes_points': PosixPath('/Users/christian/data/zooniverse/2024_04_16_analysis/Iguanas 2nd launch/flat_panoptes_points_Iguanas 2nd launch.csv'),
 'panoptes_question': Pos

# Look into the subjects file
This contains the mappings from the subject_id to the image file

In [3]:
# read the original file
df_subjects = pd.read_csv("./data/zooniverse/iguanas-from-above-subjects.csv", sep=",")



/var/folders/2k/78nn7s4548986wsjh29rhj9w0000gn/T/ipykernel_48224/3062261204.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_subjects = pd.read_csv("./data/zooniverse/iguanas-from-above-subjects.csv", sep=",")


In [4]:
# filter the subjects for only the images in the three phases

df_subjects = df_subjects[df_subjects.workflow_id.isin([workflow_id_p1, workflow_id_p2, workflow_id_p3])]


In [5]:
# inspect the metadata
import json
def get_json_keys(json_str):
    try:
        json_obj = json.loads(json_str)
        return list(json_obj.keys())
    except json.JSONDecodeError:
        return []

# Apply the function to each row in the metadata column and collect all keys
all_keys = df_subjects['locations'].apply(get_json_keys)

# Flatten the list of lists and get unique keys
unique_keys = set([key for sublist in all_keys for key in sublist])

print(unique_keys)

{'0'}


Clean up the subjects file

In [6]:
df_subjects["image_name"] = df_subjects['metadata'].apply(lambda x: json.loads(x).get('Image_name') 
                                        or json.loads(x).get('image_name') 
                                        or json.loads(x).get('Filename')).sort_values(ascending=True)

# 'site', 'flight', 'Flight', 'Site', 'flight_code' depict the same
df_subjects["flight_code"] = df_subjects['metadata'].apply(lambda x: json.loads(x).get('flight_code') 
                                        or json.loads(x).get('site') 
                                        or json.loads(x).get('flight')
                                        or json.loads(x).get('Flight')
                                        or json.loads(x).get('Site')).sort_values(ascending=True)

df_subjects["url"] = df_subjects['locations'].apply(lambda x: json.loads(x)["0"])
df_subjects["filepath"] = None

In [7]:
from loguru import logger
from time import sleep
# helper function to download the images
import requests

def download_image(url, filename):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(filename, 'wb') as file:
                file.write(response.content)
            return True
        else:
            logger.warning(f"Failed to download {url}")
            logger.error(response)
            sleep(5)
            return False
    except Exception as e:
        logger.error(e)
        sleep(5)
        return False

/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Panoptes Data Extraction from Zooniverse
## Panoptes config
### Create the configuration files automatically
The configurations were changed to custom workflow versions.

In [8]:
# create a configuration file from the workflow
#!mkdir ./data/phase_1
#! panoptes_aggregation config /Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-workflows.csv 14370 --min_version 0 --max_version 142.245 -d ./data/phase_1
# 
#!mkdir ./data/phase_2
#! panoptes_aggregation config /Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-workflows.csv 20600 --min_version 0 --max_version 94.166 -d ./data/phase_2
# 
#!mkdir ./data/phase_3
#! panoptes_aggregation config /Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-workflows.csv 22040 --min_version 0 --max_version 9.63 -d ./data/phase_3

## Extract the data

In [9]:
# phase 1
if data_folder == "./data/phase_1":
    !mkdir ./data/phase_1/V121.144
    !mkdir ./data/phase_1/V134.236
    
    !panoptes_aggregation extract /Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv -d ./data/phase_1/V121.144 ./data/phase_1/Extractor_config_workflow_14370_V121.144.yaml
    
    !panoptes_aggregation extract /Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv -d ./data/phase_1/V134.236 ./data/phase_1/Extractor_config_workflow_14370_V134.236-1.yaml


In [10]:
if data_folder == "./data/phase_2" and reprocess == True:
    # phase 2
    
    !mkdir ./data/phase_2/V89.162
    !mkdir ./data/phase_2/V93.166
    !mkdir ./data/phase_2/V94.166 
    
    !panoptes_aggregation extract /Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv -d ./data/phase_2/V89.162 ./data/phase_2/Extractor_config_workflow_20600_V89.162.yaml
    !panoptes_aggregation extract /Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv -d ./data/phase_2/V93.166 ./data/phase_2/Extractor_config_workflow_20600_V93.166.yaml
    !panoptes_aggregation extract /Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv -d ./data/phase_2/V94.166 ./data/phase_2/Extractor_config_workflow_20600_V94.166.yaml



In [11]:
if data_folder == "./data/phase_3":
    !mkdir ./data/phase_3/V7.63
    !mkdir ./data/phase_3/V9.63
    
    !panoptes_aggregation extract /Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv -d ./data/phase_3/V7.63 ./data/phase_3/Extractor_config_workflow_22040_V7.63.yaml
    !panoptes_aggregation extract /Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv -d ./data/phase_3/V9.63 ./data/phase_3/Extractor_config_workflow_22040_V9.63.yaml

### Merge the single point and questions extractions

In [12]:
# phase 1
if data_folder == "./data/phase_1":
    df_panoptes_point_extractor_1 = pd.read_csv(f"./data/phase_1/V121.144/point_extractor_by_frame_extractions.csv", sep=",")
    df_panoptes_point_extractor_2 = pd.read_csv(f"./data/phase_1/V134.236/point_extractor_by_frame_extractions.csv", sep=",")
    df_panoptes_point_extractor_1["workflow_version"] = "121.144"
    df_panoptes_point_extractor_2["workflow_version"] = "134.236"
    
    df_panoptes_question_1 = pd.read_csv(f"{data_folder}/V121.144/question_extractor_extractions.csv", sep=",")
    df_panoptes_question_2 = pd.read_csv(f"{data_folder}/V134.236/question_extractor_extractions.csv", sep=",")
    
    df_panoptes_point_extractor = pd.concat([df_panoptes_point_extractor_1, df_panoptes_point_extractor_2], axis=0)
    df_panoptes_question = pd.concat([df_panoptes_question_1, df_panoptes_question_2], axis=0)
    
    df_panoptes_point_extractor

In [ ]:
# # phase 2
if data_folder == "./data/phase_2":
    # read the rectangles annotations too there
    df_panotes_rectangle_extractor_1 = pd.read_csv(f"{data_folder}/V89.162/shape_extractor_rectangle_extractions.csv", sep=",")
    
    df_panoptes_point_extractor_1 = pd.read_csv(f"{data_folder}/V89.162/point_extractor_by_frame_extractions.csv", sep=",")
    df_panoptes_point_extractor_2 = pd.read_csv(f"{data_folder}/V93.166/point_extractor_by_frame_extractions.csv", sep=",")
    df_panoptes_point_extractor_3 = pd.read_csv(f"{data_folder}/V94.166/point_extractor_by_frame_extractions.csv", sep=",")
    
    df_panoptes_point_extractor_1["workflow_version"] = "89.162"
    df_panoptes_point_extractor_2["workflow_version"] = "93.166"
    df_panoptes_point_extractor_3["workflow_version"] = "94.166"
    
    df_panoptes_question_1 = pd.read_csv(f"{data_folder}/V89.162/question_extractor_extractions.csv", sep=",")
    df_panoptes_question_2 = pd.read_csv(f"{data_folder}/V93.166/question_extractor_extractions.csv", sep=",")
    df_panoptes_question_3 = pd.read_csv(f"{data_folder}/V94.166/question_extractor_extractions.csv", sep=",")
    
    df_panoptes_point_extractor = pd.concat([df_panoptes_point_extractor_1, df_panoptes_point_extractor_2, df_panoptes_point_extractor_3], axis=0)
    df_panoptes_question = pd.concat([df_panoptes_question_1, df_panoptes_question_2, df_panoptes_question_3], axis=0)

    df_panotes_rectangle_extractor_1

In [14]:
if data_folder == "./data/phase_3":
    df_panoptes_point_extractor_1 = pd.read_csv(f"{data_folder}/V7.63/point_extractor_by_frame_extractions.csv", sep=",")
    df_panoptes_point_extractor_2 = pd.read_csv(f"{data_folder}/V9.63/point_extractor_by_frame_extractions.csv", sep=",")
    df_panoptes_point_extractor_1["workflow_version"] = "7.63"
    df_panoptes_point_extractor_2["workflow_version"] = "9.63"

    df_panoptes_question_1 = pd.read_csv(f"{data_folder}/V7.63/question_extractor_extractions.csv", sep=",")
    df_panoptes_question_2 = pd.read_csv(f"{data_folder}/V9.63/question_extractor_extractions.csv", sep=",")

    df_panoptes_point_extractor = pd.concat([df_panoptes_point_extractor_1, df_panoptes_point_extractor_2], axis=0)
    df_panoptes_question = pd.concat([df_panoptes_question_1, df_panoptes_question_2], axis=0)


In [15]:
df_panoptes_point_extractor

,classification_id,user_name,user_id,workflow_id,task,created_at,subject_id,extractor,data.aggregation_version,data.frame0.T4_tool0_x,...,data.frame0.T4_tool6_y,workflow_version,data.frame0.T2_tool1_x,data.frame0.T2_tool1_y,data.frame0.T2_tool0_x,data.frame0.T2_tool0_y,data.frame0.T2_tool2_x,data.frame0.T2_tool2_y,data.frame0.T2_tool3_x,data.frame0.T2_tool3_y
0,393742576,AmyMacLeod,1443210.0,20600,T4,2022-02-01 00:09:54 UTC,72335168,point_extractor_by_frame,4.1.0,"[1481.24658203125, 64.83441925048828, 152.1798...",...,NaN,89.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,393742638,AmyMacLeod,1443210.0,20600,T4,2022-02-01 00:10:20 UTC,72334984,point_extractor_by_frame,4.1.0,NaN,...,NaN,89.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,393742678,AmyMacLeod,1443210.0,20600,T4,2022-02-01 00:10:33 UTC,72340244,point_extractor_by_frame,4.1.0,NaN,...,NaN,89.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,393743049,AmyMacLeod,1443210.0,20600,T4,2022-02-01 00:12:15 UTC,72336318,point_extractor_by_frame,4.1.0,NaN,...,NaN,89.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,393768668,not-logged-in-b0c587027139fe5e6549,NaN,20600,T4,2022-02-01 03:12:48 UTC,72372999,point_extractor_by_frame,4.1.0,NaN,...,NaN,89.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176631,412711830,Melisss,1369932.0,20600,T4,2022-05-03 07:08:50 UTC,72333494,point_extractor_by_frame,4.1.0,"[944.7937622070312, 1230.0899658203125]",...,NaN,94.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176632,412712258,Melisss,1369932.0,20600,T2,2022-05-03 07:12:18 UTC,72333471,point_extractor_by_frame,4.1.0,NaN,...,NaN,94.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176633,412712258,Melisss,1369932.0,20600,T4,2022-05-03 07:12:18 UTC,72333471,point_extractor_by_frame,4.1.0,[1212.106201171875],...,NaN,94.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176634,456473615,edengbn,2462144.0,20600,T2,2022-12-13 01:27:15 UTC,72341496,point_extractor_by_frame,4.1.0,NaN,...,NaN,94.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# join the image name from the subjects file
df_panoptes_point_extractor = df_panoptes_point_extractor.merge(df_subjects[["subject_id", "image_name"]], left_on="subject_id", right_on="subject_id")
df_panoptes_point_extractor = df_panoptes_point_extractor[df_panoptes_point_extractor.subject_id.isin(df_subjects.subject_id)]

df_panoptes_point_extractor

,classification_id,user_name,user_id,workflow_id,task,created_at,subject_id,extractor,data.aggregation_version,data.frame0.T4_tool0_x,...,workflow_version,data.frame0.T2_tool1_x,data.frame0.T2_tool1_y,data.frame0.T2_tool0_x,data.frame0.T2_tool0_y,data.frame0.T2_tool2_x,data.frame0.T2_tool2_y,data.frame0.T2_tool3_x,data.frame0.T2_tool3_y,image_name
0,393742576,AmyMacLeod,1443210.0,20600,T4,2022-02-01 00:09:54 UTC,72335168,point_extractor_by_frame,4.1.0,"[1481.24658203125, 64.83441925048828, 152.1798...",...,89.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EIG05-2_55.jpg
1,394132937,laswett,2309767.0,20600,T2,2022-02-02 19:19:04 UTC,72335168,point_extractor_by_frame,4.1.0,NaN,...,93.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EIG05-2_55.jpg
2,394132937,laswett,2309767.0,20600,T4,2022-02-02 19:19:04 UTC,72335168,point_extractor_by_frame,4.1.0,"[1474.2442626953125, 144.0193634033203, 12.096...",...,93.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EIG05-2_55.jpg
3,394221102,colarsoledad,2360454.0,20600,T2,2022-02-03 03:24:54 UTC,72335168,point_extractor_by_frame,4.1.0,NaN,...,93.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EIG05-2_55.jpg
4,394221102,colarsoledad,2360454.0,20600,T4,2022-02-03 03:24:54 UTC,72335168,point_extractor_by_frame,4.1.0,"[1474.424560546875, 146.7071990966797, 12.0523...",...,93.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EIG05-2_55.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392782,411516154,roserz12,2463318.0,20600,T4,2022-04-26 22:17:47 UTC,72339681,point_extractor_by_frame,4.1.0,NaN,...,94.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FPA04_31.jpg
392783,411521759,Jubumwun,2440846.0,20600,T2,2022-04-26 22:55:30 UTC,72339681,point_extractor_by_frame,4.1.0,NaN,...,94.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FPA04_31.jpg
392784,411521759,Jubumwun,2440846.0,20600,T4,2022-04-26 22:55:30 UTC,72339681,point_extractor_by_frame,4.1.0,NaN,...,94.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FPA04_31.jpg
392785,411540463,CallieSanDiego,1989263.0,20600,T2,2022-04-27 01:13:51 UTC,72339681,point_extractor_by_frame,4.1.0,NaN,...,94.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FPA04_31.jpg


## Anonymise the data

In [17]:
from hashlib import blake2b

df_panoptes_point_extractor["user_id"] = df_panoptes_point_extractor['user_id'].apply(lambda x: blake2b(str(x).encode(), digest_size=16).hexdigest() if not pd.isnull(x) else x)
# Anonymize 'user_name' by hashing
df_panoptes_point_extractor['user_name'] = df_panoptes_point_extractor['user_name'].apply(lambda x: blake2b(x.encode(), digest_size=16).hexdigest() if isinstance(x, str) else x)

df_panoptes_question["user_id"] = df_panoptes_question['user_id'].apply(lambda x: blake2b(str(x).encode(), digest_size=16).hexdigest() if not pd.isnull(x) else x)
# Anonymize 'user_name' by hashing
df_panoptes_question['user_name'] = df_panoptes_question['user_name'].apply(lambda x: blake2b(x.encode(), digest_size=16).hexdigest() if isinstance(x, str) else x)

In [18]:
df_panoptes_point_extractor

,classification_id,user_name,user_id,workflow_id,task,created_at,subject_id,extractor,data.aggregation_version,data.frame0.T4_tool0_x,...,workflow_version,data.frame0.T2_tool1_x,data.frame0.T2_tool1_y,data.frame0.T2_tool0_x,data.frame0.T2_tool0_y,data.frame0.T2_tool2_x,data.frame0.T2_tool2_y,data.frame0.T2_tool3_x,data.frame0.T2_tool3_y,image_name
0,393742576,19835766ecbb50ec6fbe68caa85e4c63,8bfe78a8dc879fc7aabe1ff94a42d74f,20600,T4,2022-02-01 00:09:54 UTC,72335168,point_extractor_by_frame,4.1.0,"[1481.24658203125, 64.83441925048828, 152.1798...",...,89.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EIG05-2_55.jpg
1,394132937,691500ccebe2131f83809524df652f87,c11a32c827347926881e5e1db75cb701,20600,T2,2022-02-02 19:19:04 UTC,72335168,point_extractor_by_frame,4.1.0,NaN,...,93.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EIG05-2_55.jpg
2,394132937,691500ccebe2131f83809524df652f87,c11a32c827347926881e5e1db75cb701,20600,T4,2022-02-02 19:19:04 UTC,72335168,point_extractor_by_frame,4.1.0,"[1474.2442626953125, 144.0193634033203, 12.096...",...,93.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EIG05-2_55.jpg
3,394221102,07452636222fb135e92fd5bedbecd18c,1dcb5107d19f3fa3d0d40eb4fa5a32ac,20600,T2,2022-02-03 03:24:54 UTC,72335168,point_extractor_by_frame,4.1.0,NaN,...,93.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EIG05-2_55.jpg
4,394221102,07452636222fb135e92fd5bedbecd18c,1dcb5107d19f3fa3d0d40eb4fa5a32ac,20600,T4,2022-02-03 03:24:54 UTC,72335168,point_extractor_by_frame,4.1.0,"[1474.424560546875, 146.7071990966797, 12.0523...",...,93.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EIG05-2_55.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392782,411516154,9ab0dec08676f4d8147dc2795c832b1c,9ca59dc6cdb395496bace4f9b9bb887b,20600,T4,2022-04-26 22:17:47 UTC,72339681,point_extractor_by_frame,4.1.0,NaN,...,94.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FPA04_31.jpg
392783,411521759,21f1cdf48bb3a0f13deeb1de0cfe194a,db20ef8c2ff775170aa11a62f2a288da,20600,T2,2022-04-26 22:55:30 UTC,72339681,point_extractor_by_frame,4.1.0,NaN,...,94.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FPA04_31.jpg
392784,411521759,21f1cdf48bb3a0f13deeb1de0cfe194a,db20ef8c2ff775170aa11a62f2a288da,20600,T4,2022-04-26 22:55:30 UTC,72339681,point_extractor_by_frame,4.1.0,NaN,...,94.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FPA04_31.jpg
392785,411540463,14d24a5c9234b322cb10145547ea5222,aa43b1f9c4c4b98e2457b3c5bc1690c8,20600,T2,2022-04-27 01:13:51 UTC,72339681,point_extractor_by_frame,4.1.0,NaN,...,94.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FPA04_31.jpg


## Determine the amount of yes Answers for "Is there an Iguana"

In [19]:
df_panoptes_question

,classification_id,user_name,user_id,workflow_id,task,created_at,subject_id,extractor,data.no,data.aggregation_version,data.yes
0,393742576,19835766ecbb50ec6fbe68caa85e4c63,8bfe78a8dc879fc7aabe1ff94a42d74f,20600,T0,2022-02-01 00:09:54 UTC,72335168,question_extractor,1.0,4.1.0,NaN
1,393742638,19835766ecbb50ec6fbe68caa85e4c63,8bfe78a8dc879fc7aabe1ff94a42d74f,20600,T0,2022-02-01 00:10:20 UTC,72334984,question_extractor,1.0,4.1.0,NaN
2,393742678,19835766ecbb50ec6fbe68caa85e4c63,8bfe78a8dc879fc7aabe1ff94a42d74f,20600,T0,2022-02-01 00:10:33 UTC,72340244,question_extractor,1.0,4.1.0,NaN
3,393743049,19835766ecbb50ec6fbe68caa85e4c63,8bfe78a8dc879fc7aabe1ff94a42d74f,20600,T0,2022-02-01 00:12:15 UTC,72336318,question_extractor,NaN,4.1.0,1.0
4,393768668,dc0ea5c32da0e322969ff6e6aa76ca5f,NaN,20600,T0,2022-02-01 03:12:48 UTC,72372999,question_extractor,NaN,4.1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
88313,412711106,d607a725449d5e3a2deaba5014fce17d,dabdbb41b2897bd5fdef6dc4ca74584b,20600,T0,2022-05-03 07:02:18 UTC,72333409,question_extractor,1.0,4.1.0,NaN
88314,412711577,d607a725449d5e3a2deaba5014fce17d,dabdbb41b2897bd5fdef6dc4ca74584b,20600,T0,2022-05-03 07:06:48 UTC,72333423,question_extractor,1.0,4.1.0,NaN
88315,412711830,d607a725449d5e3a2deaba5014fce17d,dabdbb41b2897bd5fdef6dc4ca74584b,20600,T0,2022-05-03 07:08:50 UTC,72333494,question_extractor,1.0,4.1.0,NaN
88316,412712258,d607a725449d5e3a2deaba5014fce17d,dabdbb41b2897bd5fdef6dc4ca74584b,20600,T0,2022-05-03 07:12:18 UTC,72333471,question_extractor,1.0,4.1.0,NaN


In [20]:
df_panoptes_question_r = df_panoptes_question[df_panoptes_question.task == "T0"][["subject_id", "data.no", "data.yes"]].groupby("subject_id").sum()

df_panoptes_question_r = df_panoptes_question_r.reset_index()
df_panoptes_question_r = df_panoptes_question_r[df_panoptes_question_r.subject_id.isin(df_subjects.subject_id)]
df_panoptes_question_r

,subject_id,data.no,data.yes
0,72332768,22.0,0.0
1,72332769,21.0,0.0
2,72332770,21.0,0.0
3,72332771,20.0,1.0
4,72332772,21.0,2.0
...,...,...,...
9092,72373345,19.0,2.0
9093,72373347,18.0,4.0
9094,72373348,5.0,16.0
9095,72373350,6.0,15.0


In [21]:
df_panoptes_question_r.to_csv(output_path / config["panoptes_question"], index = False)

## Get the Point Marks Analysis Ready

Filter for T2 only

In [22]:
df_panoptes_point_extractor_r = df_panoptes_point_extractor[
    (df_panoptes_point_extractor.task == "T2")
]
df_panoptes_point_extractor_r.columns

Index(['classification_id', 'user_name', 'user_id', 'workflow_id', 'task',
       'created_at', 'subject_id', 'extractor', 'data.aggregation_version',
       'data.frame0.T4_tool0_x', 'data.frame0.T4_tool0_y',
       'data.frame0.T4_tool2_x', 'data.frame0.T4_tool2_y',
       'data.frame0.T4_tool3_x', 'data.frame0.T4_tool3_y',
       'data.frame0.T4_tool1_x', 'data.frame0.T4_tool1_y',
       'data.frame0.T4_tool4_x', 'data.frame0.T4_tool4_y',
       'data.frame0.T4_tool5_x', 'data.frame0.T4_tool5_y',
       'data.frame0.T4_tool7_x', 'data.frame0.T4_tool7_y',
       'data.frame0.T4_tool6_x', 'data.frame0.T4_tool6_y', 'workflow_version',
       'data.frame0.T2_tool1_x', 'data.frame0.T2_tool1_y',
       'data.frame0.T2_tool0_x', 'data.frame0.T2_tool0_y',
       'data.frame0.T2_tool2_x', 'data.frame0.T2_tool2_y',
       'data.frame0.T2_tool3_x', 'data.frame0.T2_tool3_y', 'image_name'],
      dtype='object')

### Which tool is which now?
| Tool Name               | Classification                               |
|-------------------------|----------------------------------------------|
| data.frame0.T2_tool0_x  | Adult Male in a lek                          |
| data.frame0.T2_tool1_x  | Adult Male alone                             |
| data.frame0.T2_tool2_x  | Others (females, young males, juveniles)     |
| data.frame0.T2_tool3_x  | Partial iguana                               |
| data.frame0.T2_tool4_x  | Could be an iguana, not sure                 |

Is "Could be an iguana, not sure" and "Partial Iguana" are omitted.


In [23]:
# create a flat structure from the nested marks over multiple columns from that.
from ast import literal_eval

columns_keep_x = ['data.frame0.T2_tool0_x', 'data.frame0.T2_tool1_x', 'data.frame0.T2_tool2_x']
columns_keep_y = ['data.frame0.T2_tool0_y', 'data.frame0.T2_tool1_y', 'data.frame0.T2_tool2_y']

for col in columns_keep_x + columns_keep_y:
    df_panoptes_point_extractor_r[col] = df_panoptes_point_extractor_r[col].apply(lambda x: literal_eval(x) if pd.notnull(x) else [])

# Merge the lists in 'x' and 'y' coordinates
df_panoptes_point_extractor_r['x'] = df_panoptes_point_extractor_r[columns_keep_x].values.tolist()
df_panoptes_point_extractor_r['y'] = df_panoptes_point_extractor_r[columns_keep_y].values.tolist()

# Flatten the lists in each row for 'x' and 'y'
df_panoptes_point_extractor_r['x'] = df_panoptes_point_extractor_r['x'].apply(lambda x: [item for sublist in x for item in sublist])
df_panoptes_point_extractor_r['y'] = df_panoptes_point_extractor_r['y'].apply(lambda x: [item for sublist in x for item in sublist])

# Explode the DataFrame to separate rows for each x, y pair
# Note: This requires pandas >= 0.25 for simultaneous explode
# df_panoptes_point_extractor_r_exploded = df_panoptes_point_extractor_r.apply(pd.Series.explode)

# Explode the DataFrame based on these columns to get separate rows for each list element
# Make sure to perform the explode operation on both columns simultaneously to keep the x and y coordinates paired
df_panoptes_point_extractor_r

/var/folders/2k/78nn7s4548986wsjh29rhj9w0000gn/T/ipykernel_48224/2729049416.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_panoptes_point_extractor_r[col] = df_panoptes_point_extractor_r[col].apply(lambda x: literal_eval(x) if pd.notnull(x) else [])
/var/folders/2k/78nn7s4548986wsjh29rhj9w0000gn/T/ipykernel_48224/2729049416.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_panoptes_point_extractor_r['x'] = df_panoptes_point_extractor_r[columns_keep_x].values.tolist()
/var/folders/2k/78nn7s4548

,classification_id,user_name,user_id,workflow_id,task,created_at,subject_id,extractor,data.aggregation_version,data.frame0.T4_tool0_x,...,data.frame0.T2_tool1_y,data.frame0.T2_tool0_x,data.frame0.T2_tool0_y,data.frame0.T2_tool2_x,data.frame0.T2_tool2_y,data.frame0.T2_tool3_x,data.frame0.T2_tool3_y,image_name,x,y
1,394132937,691500ccebe2131f83809524df652f87,c11a32c827347926881e5e1db75cb701,20600,T2,2022-02-02 19:19:04 UTC,72335168,point_extractor_by_frame,4.1.0,NaN,...,[],[],[],[],[],NaN,NaN,EIG05-2_55.jpg,[],[]
3,394221102,07452636222fb135e92fd5bedbecd18c,1dcb5107d19f3fa3d0d40eb4fa5a32ac,20600,T2,2022-02-03 03:24:54 UTC,72335168,point_extractor_by_frame,4.1.0,NaN,...,[],[],[],[],[],NaN,NaN,EIG05-2_55.jpg,[],[]
5,394417706,51ff8f6bdcfbc629ee64b0f03456f9be,73e55824a8121ac5c7e625d8644450fc,20600,T2,2022-02-03 22:37:12 UTC,72335168,point_extractor_by_frame,4.1.0,NaN,...,[],[],[],[],[],NaN,NaN,EIG05-2_55.jpg,[],[]
7,394681993,1f1fbd60a47a8f619cab23b0367b305c,a0763c2de16d5ffb4cca7e97cfdba769,20600,T2,2022-02-05 00:53:57 UTC,72335168,point_extractor_by_frame,4.1.0,NaN,...,[],[],[],[],[],NaN,NaN,EIG05-2_55.jpg,[],[]
9,394699098,b9dfe23678c9bbf754d04706a6e36269,a2f79791c7443eb1b6fec361a61f896e,20600,T2,2022-02-05 03:46:10 UTC,72335168,point_extractor_by_frame,4.1.0,NaN,...,[],[],[],[],[],NaN,NaN,EIG05-2_55.jpg,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392777,411445399,c78b539934f9c55f492e674b69e237c0,NaN,20600,T2,2022-04-26 17:54:00 UTC,72339681,point_extractor_by_frame,4.1.0,NaN,...,[],[],[],[],[],NaN,NaN,FPA04_31.jpg,[],[]
392779,411502672,097fcaf77e79d9ac515e3638cdf35ccd,c776f96dfe44a1bb1bf2f6e137ea40ea,20600,T2,2022-04-26 21:09:05 UTC,72339681,point_extractor_by_frame,4.1.0,NaN,...,[],[],[],[],[],NaN,NaN,FPA04_31.jpg,[],[]
392781,411516154,9ab0dec08676f4d8147dc2795c832b1c,9ca59dc6cdb395496bace4f9b9bb887b,20600,T2,2022-04-26 22:17:47 UTC,72339681,point_extractor_by_frame,4.1.0,NaN,...,[],[],[],[],[],NaN,NaN,FPA04_31.jpg,[],[]
392783,411521759,21f1cdf48bb3a0f13deeb1de0cfe194a,db20ef8c2ff775170aa11a62f2a288da,20600,T2,2022-04-26 22:55:30 UTC,72339681,point_extractor_by_frame,4.1.0,NaN,...,[],[],[],[],[],NaN,NaN,FPA04_31.jpg,[],[]


In [24]:
df_panoptes_point_extractor_r = df_panoptes_point_extractor_r[
    ['classification_id', 'user_name', 'user_id', 'workflow_id',  'workflow_version', 'task',
     'created_at', 'subject_id', "image_name",
     'x', 'y'
     ]].reset_index(drop=True)

df_panoptes_point_extractor_r

,classification_id,user_name,user_id,workflow_id,workflow_version,task,created_at,subject_id,image_name,x,y
0,394132937,691500ccebe2131f83809524df652f87,c11a32c827347926881e5e1db75cb701,20600,93.166,T2,2022-02-02 19:19:04 UTC,72335168,EIG05-2_55.jpg,[],[]
1,394221102,07452636222fb135e92fd5bedbecd18c,1dcb5107d19f3fa3d0d40eb4fa5a32ac,20600,93.166,T2,2022-02-03 03:24:54 UTC,72335168,EIG05-2_55.jpg,[],[]
2,394417706,51ff8f6bdcfbc629ee64b0f03456f9be,73e55824a8121ac5c7e625d8644450fc,20600,93.166,T2,2022-02-03 22:37:12 UTC,72335168,EIG05-2_55.jpg,[],[]
3,394681993,1f1fbd60a47a8f619cab23b0367b305c,a0763c2de16d5ffb4cca7e97cfdba769,20600,93.166,T2,2022-02-05 00:53:57 UTC,72335168,EIG05-2_55.jpg,[],[]
4,394699098,b9dfe23678c9bbf754d04706a6e36269,a2f79791c7443eb1b6fec361a61f896e,20600,93.166,T2,2022-02-05 03:46:10 UTC,72335168,EIG05-2_55.jpg,[],[]
...,...,...,...,...,...,...,...,...,...,...,...
196125,411445399,c78b539934f9c55f492e674b69e237c0,NaN,20600,94.166,T2,2022-04-26 17:54:00 UTC,72339681,FPA04_31.jpg,[],[]
196126,411502672,097fcaf77e79d9ac515e3638cdf35ccd,c776f96dfe44a1bb1bf2f6e137ea40ea,20600,94.166,T2,2022-04-26 21:09:05 UTC,72339681,FPA04_31.jpg,[],[]
196127,411516154,9ab0dec08676f4d8147dc2795c832b1c,9ca59dc6cdb395496bace4f9b9bb887b,20600,94.166,T2,2022-04-26 22:17:47 UTC,72339681,FPA04_31.jpg,[],[]
196128,411521759,21f1cdf48bb3a0f13deeb1de0cfe194a,db20ef8c2ff775170aa11a62f2a288da,20600,94.166,T2,2022-04-26 22:55:30 UTC,72339681,FPA04_31.jpg,[],[]


In [25]:
# explode the lists of marks per user into one row per mark
df_panoptes_point_extractor_r_ex = df_panoptes_point_extractor_r.apply(lambda x: x.explode() if x.name in ['x', 'y'] else x)

In [26]:
# images with no marks have NaN values in the 'merged_x' and 'merged_y' columns
df_panoptes_point_extractor_r_ex_dropped = df_panoptes_point_extractor_r_ex.dropna(subset=['x', 'y'], how='all').sort_values(by=['user_id', 'subject_id', 'task', 'created_at'])
df_panoptes_point_extractor_r_ex_dropped

,classification_id,user_name,user_id,workflow_id,workflow_version,task,created_at,subject_id,image_name,x,y
131510,406989334,da233c785cfd7c42279d2f2b632bbc4f,001d00e0739694888f7dc3471fc76e5c,20600,94.166,T2,2022-04-04 00:07:09 UTC,72335718,EPS04-1_50.jpg,557.795471,529.253357
131510,406989334,da233c785cfd7c42279d2f2b632bbc4f,001d00e0739694888f7dc3471fc76e5c,20600,94.166,T2,2022-04-04 00:07:09 UTC,72335718,EPS04-1_50.jpg,604.893799,472.164551
131510,406989334,da233c785cfd7c42279d2f2b632bbc4f,001d00e0739694888f7dc3471fc76e5c,20600,94.166,T2,2022-04-04 00:07:09 UTC,72335718,EPS04-1_50.jpg,1068.740234,917.457214
180711,406983766,da233c785cfd7c42279d2f2b632bbc4f,001d00e0739694888f7dc3471fc76e5c,20600,94.166,T2,2022-04-03 23:26:39 UTC,72335747,EPS04-1_87.jpg,116.374191,815.917847
180711,406983766,da233c785cfd7c42279d2f2b632bbc4f,001d00e0739694888f7dc3471fc76e5c,20600,94.166,T2,2022-04-03 23:26:39 UTC,72335747,EPS04-1_87.jpg,173.308746,850.901733
...,...,...,...,...,...,...,...,...,...,...,...
94553,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,93.166,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,97.770393,117.218422
94553,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,93.166,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,113.634987,186.909332
94553,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,93.166,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,87.571724,167.078568
94553,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,93.166,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,443.786987,124.978645


In [27]:
# cast x and y to int
df_panoptes_point_extractor_r_ex_dropped = df_panoptes_point_extractor_r_ex_dropped.astype({'x': 'int32', 'y': 'int32'})
df_panoptes_point_extractor_r_ex_dropped

,classification_id,user_name,user_id,workflow_id,workflow_version,task,created_at,subject_id,image_name,x,y
131510,406989334,da233c785cfd7c42279d2f2b632bbc4f,001d00e0739694888f7dc3471fc76e5c,20600,94.166,T2,2022-04-04 00:07:09 UTC,72335718,EPS04-1_50.jpg,557,529
131510,406989334,da233c785cfd7c42279d2f2b632bbc4f,001d00e0739694888f7dc3471fc76e5c,20600,94.166,T2,2022-04-04 00:07:09 UTC,72335718,EPS04-1_50.jpg,604,472
131510,406989334,da233c785cfd7c42279d2f2b632bbc4f,001d00e0739694888f7dc3471fc76e5c,20600,94.166,T2,2022-04-04 00:07:09 UTC,72335718,EPS04-1_50.jpg,1068,917
180711,406983766,da233c785cfd7c42279d2f2b632bbc4f,001d00e0739694888f7dc3471fc76e5c,20600,94.166,T2,2022-04-03 23:26:39 UTC,72335747,EPS04-1_87.jpg,116,815
180711,406983766,da233c785cfd7c42279d2f2b632bbc4f,001d00e0739694888f7dc3471fc76e5c,20600,94.166,T2,2022-04-03 23:26:39 UTC,72335747,EPS04-1_87.jpg,173,850
...,...,...,...,...,...,...,...,...,...,...,...
94553,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,93.166,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,97,117
94553,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,93.166,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,113,186
94553,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,93.166,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,87,167
94553,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,93.166,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,443,124


In [28]:
df_panoptes_point_extractor_r_ex_dropped.to_csv(config["flat_panoptes_points"], sep=",", index = False)

## Inspecting the results
Check the numbers for a single subject_id

In [29]:
### Looks the images in question

subject_id_2 = 72373250 
df_debug = df_panoptes_point_extractor_r_ex_dropped[(df_panoptes_point_extractor_r_ex_dropped.subject_id == subject_id_2)]
df_debug

,classification_id,user_name,user_id,workflow_id,workflow_version,task,created_at,subject_id,image_name,x,y
55082,408899097,dcaeb0f97c49f8f7fd4bd7778a65161b,0156eebf62383fedf03616142d065d39,20600,94.166,T2,2022-04-13 12:30:08 UTC,72373250,ESCG02-1_19.jpg,136,503
55082,408899097,dcaeb0f97c49f8f7fd4bd7778a65161b,0156eebf62383fedf03616142d065d39,20600,94.166,T2,2022-04-13 12:30:08 UTC,72373250,ESCG02-1_19.jpg,271,339
55082,408899097,dcaeb0f97c49f8f7fd4bd7778a65161b,0156eebf62383fedf03616142d065d39,20600,94.166,T2,2022-04-13 12:30:08 UTC,72373250,ESCG02-1_19.jpg,577,305
55082,408899097,dcaeb0f97c49f8f7fd4bd7778a65161b,0156eebf62383fedf03616142d065d39,20600,94.166,T2,2022-04-13 12:30:08 UTC,72373250,ESCG02-1_19.jpg,624,463
55082,408899097,dcaeb0f97c49f8f7fd4bd7778a65161b,0156eebf62383fedf03616142d065d39,20600,94.166,T2,2022-04-13 12:30:08 UTC,72373250,ESCG02-1_19.jpg,94,671
...,...,...,...,...,...,...,...,...,...,...,...
55084,409860945,c6d66b18814c9df1833d0a281e272323,NaN,20600,94.166,T2,2022-04-18 23:19:43 UTC,72373250,ESCG02-1_19.jpg,193,295
55084,409860945,c6d66b18814c9df1833d0a281e272323,NaN,20600,94.166,T2,2022-04-18 23:19:43 UTC,72373250,ESCG02-1_19.jpg,199,127
55084,409860945,c6d66b18814c9df1833d0a281e272323,NaN,20600,94.166,T2,2022-04-18 23:19:43 UTC,72373250,ESCG02-1_19.jpg,193,148
55084,409860945,c6d66b18814c9df1833d0a281e272323,NaN,20600,94.166,T2,2022-04-18 23:19:43 UTC,72373250,ESCG02-1_19.jpg,427,995


In [30]:
df_debug.groupby('user_name').size()


user_name
02e6f61e89e91b8725718f54a1ee7c56    42
14d24a5c9234b322cb10145547ea5222    35
1fdecb6e31d379d15525c6962dda8af1    32
41415b12a7025ce6d450c1ee1c175909    31
46232f4fb2d15df64b9156e591ccda6a    26
565038ede52e5fce582e210f6b806ff9    30
691500ccebe2131f83809524df652f87    36
77fedf9ed047493cad21f45b5bccbe83    29
7da528cc567b4f5ceec05c6a0de72a04    29
8405b5369ddfd9c86ad032a0bc48dcbc    35
8e05b15d5b3df382ab3b7ef725032979    28
adaaf6f43b5a95c1f5831f0462f369f5    37
b0f5fa7789909ee79b0264f924b28554    49
b351d9ed4d01ea5b3109e04ff020552d    31
b9fe6f3e7007b28548521cfc5a325cb3    21
c6d66b18814c9df1833d0a281e272323    23
d4be5c8ad02f70ce96e5daab37f25419    14
d4db022c6b48b830d8f92d5c737d6364     1
dcaeb0f97c49f8f7fd4bd7778a65161b    30
fb791d53c56a36e057a53ee561f2664e    39
dtype: int64

In [31]:
df_debug[df_debug.user_name == "CallieSanDiego"]

,classification_id,user_name,user_id,workflow_id,workflow_version,task,created_at,subject_id,image_name,x,y


## Download images
iguanas-from-above-subjects_with_url.csv will be used to track which url was already downlaoded.

In [32]:
## save the file the extra columns we need for downloading.
df_subjects.to_csv(output_path / "iguanas-from-above-subjects_with_url.csv")


# read the modified csv
df_subjects = pd.read_csv(output_path / "iguanas-from-above-subjects_with_url.csv")
df_subjects

,Unnamed: 0,subject_id,project_id,workflow_id,subject_set_id,metadata,locations,classifications_count,retired_at,retirement_reason,created_at,updated_at,image_name,flight_code,url,filepath
0,190,47967468,11905,14370.0,86008,"{""site"":""SFB"",""image_name"":""SFB01-3_08.jpg"",""s...","{""0"":""https://panoptes-uploads.zooniverse.org/...",20,2020-11-15 19:06:16 UTC,classification_count,2020-07-18 20:38:14 UTC,2020-07-18 20:38:14 UTC,SFB01-3_08.jpg,SFB,https://panoptes-uploads.zooniverse.org/subjec...,NaN
1,191,47967469,11905,14370.0,86008,"{""site"":""SFB"",""image_name"":""SFB01-3_15.jpg"",""s...","{""0"":""https://panoptes-uploads.zooniverse.org/...",20,2020-10-28 19:25:18 UTC,classification_count,2020-07-18 20:38:17 UTC,2020-07-18 20:38:17 UTC,SFB01-3_15.jpg,SFB,https://panoptes-uploads.zooniverse.org/subjec...,NaN
2,192,47967470,11905,14370.0,86008,"{""site"":""SFB"",""image_name"":""SFB01-3_27.jpg"",""s...","{""0"":""https://panoptes-uploads.zooniverse.org/...",20,2020-11-14 10:07:19 UTC,classification_count,2020-07-18 20:38:18 UTC,2020-07-18 20:38:18 UTC,SFB01-3_27.jpg,SFB,https://panoptes-uploads.zooniverse.org/subjec...,NaN
3,193,47967471,11905,14370.0,86008,"{""site"":""SFB"",""image_name"":""SFB01-3_28.jpg"",""s...","{""0"":""https://panoptes-uploads.zooniverse.org/...",20,2020-11-09 10:36:02 UTC,classification_count,2020-07-18 20:38:20 UTC,2020-07-18 20:38:20 UTC,SFB01-3_28.jpg,SFB,https://panoptes-uploads.zooniverse.org/subjec...,NaN
4,194,47967472,11905,14370.0,86008,"{""site"":""SFB"",""image_name"":""SFB01-3_34.jpg"",""s...","{""0"":""https://panoptes-uploads.zooniverse.org/...",20,2020-11-18 20:44:36 UTC,classification_count,2020-07-18 20:38:22 UTC,2020-07-18 20:38:22 UTC,SFB01-3_34.jpg,SFB,https://panoptes-uploads.zooniverse.org/subjec...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57833,58027,78965182,11905,22040.0,106640,"{""id"":""476"",""set"":""SouthCoastH"",""site"":""SouthC...","{""0"":""https://panoptes-uploads.zooniverse.org/...",30,2023-06-04 20:04:00 UTC,classification_count,2022-07-24 11:23:51 UTC,2022-07-24 11:23:51 UTC,ESCH02-2_80.jpg,SouthCoastH,https://panoptes-uploads.zooniverse.org/subjec...,NaN
57834,58028,78965183,11905,22040.0,106640,"{""id"":""477"",""set"":""SouthCoastH"",""site"":""SouthC...","{""0"":""https://panoptes-uploads.zooniverse.org/...",24,2023-08-30 16:43:06 UTC,classification_count,2022-07-24 11:23:51 UTC,2023-07-28 07:13:45 UTC,ESCH02-2_81.jpg,SouthCoastH,https://panoptes-uploads.zooniverse.org/subjec...,NaN
57835,58029,78965184,11905,22040.0,106640,"{""id"":""478"",""set"":""SouthCoastH"",""site"":""SouthC...","{""0"":""https://panoptes-uploads.zooniverse.org/...",30,2023-07-14 13:55:55 UTC,classification_count,2022-07-24 11:23:52 UTC,2022-07-24 11:23:52 UTC,ESCH02-2_91.jpg,SouthCoastH,https://panoptes-uploads.zooniverse.org/subjec...,NaN
57836,58030,78965185,11905,22040.0,106640,"{""id"":""479"",""set"":""SouthCoastH"",""site"":""SouthC...","{""0"":""https://panoptes-uploads.zooniverse.org/...",30,2023-07-20 04:37:58 UTC,classification_count,2022-07-24 11:23:53 UTC,2022-07-24 11:23:53 UTC,ESCH02-2_92.jpg,SouthCoastH,https://panoptes-uploads.zooniverse.org/subjec...,NaN


In [33]:
# df_subjects = pd.read_csv(output_path / "iguanas-from-above-subjects_with_url.csv")

# downoaded_images_path = Path("./data/downloaded_images")
# downoaded_images_path.mkdir(exist_ok=True, parents=True)
# return_val = True
# # df = df_subjects[df_subjects.subject_id.isin([44660616, 47968406])]
# # df = df_subjects[df_subjects.subject_id.isin([44660616, 47968406])]
# for index, row in df_subjects[df_subjects.workflow_id.isin([workflow_id_p1])].iterrows():
#     # Only download if necessary
#     if pd.isna(row.get("filepath")) or not row.get("filepath", False):
#         flight_code = row['flight_code']
#         url = row['url']
#         image_name = Path(row['image_name']).name
#         # Extract the filename from the URL and create a unique name using index
#         filename = downoaded_images_path.joinpath(f"{image_name}_{row['subject_id']}_{flight_code}.jpeg")
#         df_subjects.loc[index, 'filepath'] = filename
#         # Download the image
#         return_val = download_image(url, filename)
# 
#         # print(f"Downloaded {filename}")
#     if return_val == False:
#         print("there was a problem")
#         # break
        

In [34]:
df_subjects.to_csv(output_path / "iguanas-from-above-subjects_with_url.csv")